<a href="https://colab.research.google.com/github/Ixiondbz/CSE499/blob/main/Main_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install -U plotly

import json
from pathlib import Path
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm import tqdm
import pandas as pd
import random
import cv2

     |████████████████████████████████| 23.9 MB 13 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [3]:
from google.colab import drive
drive.mount('/drive')
folder = "MyDrive/MIDOG_Challenge"
midog_folder = Path("/drive") / Path(folder)

Mounted at /drive


In [ ]:
print(list(midog_folder.glob("*.*")))

In [ ]:
!apt-get install python3-openslide
from openslide import open_slide

In [6]:
image_folder = midog_folder / "images"

hamamatsu_rx_ids = list(range(0, 51))
hamamatsu_360_ids = list(range(51, 101))
aperio_ids = list(range(101, 151))
leica_ids = list(range(151, 201))

In [8]:
annotation_file = midog_folder / "MIDOG.json"
rows = []
with open(annotation_file) as f:
    data = json.load(f)

  
    categories = {1: 'mitotic figure', 2: 'hard negative'}

    for row in data["images"]:
        file_name = row["file_name"]
        image_id = row["id"]
        width = row["width"]
        height = row["height"]

        scanner  = "Hamamatsu XR"
        if image_id in hamamatsu_360_ids:
            scanner  = "Hamamatsu S360"
        if image_id in aperio_ids:
            scanner  = "Aperio CS"
        if image_id in leica_ids:
            scanner  = "Leica GT450"
         
        for annotation in [anno for anno in data['annotations'] if anno["image_id"] == image_id]:
            box = annotation["bbox"]
            cat = categories[annotation["category_id"]]

            rows.append([file_name, image_id, width, height, box, cat, scanner])

df = pd.DataFrame(rows, columns=["file_name", "image_id", "width", "height", "box", "cat", "scanner"])
df.head()

,file_name,image_id,width,height,box,cat,scanner
0,001.tiff,1,7215,5412,"[4336, 346, 4386, 396]",hard negative,Hamamatsu XR
1,001.tiff,1,7215,5412,"[756, 872, 806, 922]",hard negative,Hamamatsu XR
2,001.tiff,1,7215,5412,"[270, 4044, 320, 4094]",hard negative,Hamamatsu XR
3,001.tiff,1,7215,5412,"[6672.5, 706.5, 6722.5, 756.5]",hard negative,Hamamatsu XR
4,002.tiff,2,7215,5412,"[1872, 319, 1922, 369]",hard negative,Hamamatsu XR


In [9]:
pie_df = pd.DataFrame([
          ["mitotic figure", len(df[df["cat"] == "mitotic figure"])],
          ["hard negative", len(df[df["cat"] == "hard negative"])]], columns=["cat", "total"])

fig = px.pie(pie_df, values='total', names='cat', title='Mitotic figures vs hard negatives')
fig.show()

In [10]:
for scanner in df['scanner'].unique():

    scanner_annos = df[df['scanner'] == scanner]

    row = []
    for image_id in scanner_annos["image_id"].unique():
        image_annos = scanner_annos[scanner_annos["image_id"] == image_id]
        row.append([image_id, len(image_annos[image_annos['cat'] == 'mitotic figure']), "mitotic figure"])
        row.append([image_id, len(image_annos[image_annos['cat'] == 'hard negative']), "hard negative"])

    scanner_meta = pd.DataFrame(row, columns=["image_id", "total", "type"])

    fig = px.bar(scanner_meta, x="image_id", y="total", color="type", title=f"{scanner}: Annotations per image")
    fig.show()